In [1]:
import ipywidgets as widgets
from IPython.display import display, clear_output
from pyomo.opt import SolverFactory, TerminationCondition
from src.analyses.value_function import ValueFunction
from src.analyses.convergence_analysis import HausdorffAnalyzer
import numpy as np
import matplotlib.pyplot as plt
from src.models.cz_model import CaoZavalaModel, CaoZavalaAlgo
from scipy.optimize import curve_fit
from NSPLIB.src.instances.pooling_contract_selection.pooling import const_model

# Constants
pooling_obj = -1338.2471283376406
pooling_sol = {
    'A[1]': 300.0, 'A[2]': 201.92127476313524, 'A[3]': 0.0, 'A[4]': 0.0, 'A[5]': 245.18105081826008,
    'S[1]': 247.10232558139526, 'S[2]': 0.0, 'S[3]': 0.0, 'S[4]': 500.0
}

pooling_y_bound = {
    'A[1]': [0, 300], 'A[2]': [0, 250], 'A[3]': [0, 0], 'A[4]': [0, 0], 'A[5]': [0, 300],
    'S[1]': [0, 400], 'S[2]': [0, 0], 'S[3]': [0, 0], 'S[4]': [0, 500]
}
pooling_y_bound = {
    'A[1]': [290, 300], 'A[2]': [190, 210], 'A[3]': [0, 0], 'A[4]': [0, 0], 'A[5]': [240, 260],
    'S[1]': [230, 250], 'S[2]': [0, 0], 'S[3]': [0, 0], 'S[4]': [490, 500]
}
# Build model
sto_m = const_model()
m = CaoZavalaModel.from_sto_m(sto_m)
m.build()

# Fix binary variables
binary_ys = ['lambd[1]', 'lambd[2]', 'lambd[3]', 'lambd[4]', 'lambd[5]', 'theta[1]', 'theta[2]', 'theta[3]', 'theta[4]']
binary_y_val = {y: 0 for y in binary_ys}
binary_y_val['lambd[1]'] = 1
binary_y_val['lambd[2]'] = 1
binary_y_val['lambd[5]'] = 1
binary_y_val['theta[1]'] = 1
binary_y_val['theta[4]'] = 1
m.fix_binary_y(binary_y_val)

v_f= ValueFunction(m,solver="baron")
y_dimension = {'A[1]':{}, 'A[2]':{}, 'A[5]':{},'S[1]':{}, 'S[4]':{}}
steps=10
for y in y_dimension.keys():
    v_f.calc_1D(idx=y, y_val_fix=pooling_sol,
                    interval=pooling_y_bound[y], step=steps)
    for s in m.scenarios:
        y_dimension[y][s] = v_f.value_func[s]
    y_dimension[y]['total'] = v_f.total_value_func


# Specify grid dimensions
n_rows = len(y_dimension)
n_cols = len(m.scenarios) + 1  # +1 for 'total'

# Create a grid of Output widgets for the plots
outputs = [[widgets.Output() for _ in range(n_cols)] for _ in range(n_rows)]

# Fill each plot with the corresponding value function data
for i, y_key in enumerate(y_dimension.keys()):
    for j in range(n_cols):
        with outputs[i][j]:
            x = np.linspace(pooling_y_bound[y_key][0], pooling_y_bound[y_key][1], len(y_dimension[y_key]['total']))
            if j < len(m.scenarios):
                scenario_key = list(m.scenarios)[j]
                y_vals = y_dimension[y_key][scenario_key]
                title = f"{y_key} - Scenario {scenario_key}"
            else:
                y_vals = y_dimension[y_key]['total']
                title = f"{y_key} - Total"
            fig, ax = plt.subplots(figsize=(4,4))
            ax.plot(x, y_vals)
            ax.set_title(title)
            ax.set_xlabel(y_key)
            ax.set_ylabel("Objective Value")
            ax.tick_params(axis='both', labelsize=14)
            ax.xaxis.label.set_size(16)
            ax.yaxis.label.set_size(16)
            ax.title.set_size(18)
            plt.tight_layout()
            plt.show()

# Arrange the outputs in a grid
grid = widgets.GridBox(
    [outputs[i][j] for i in range(n_rows) for j in range(n_cols)],
    layout=widgets.Layout(grid_template_columns=f"repeat({n_cols}, 400px)")
)

display(grid)

GridBox(children=(Output(), Output(), Output(), Output(), Output(), Output(), Output(), Output(), Output(), Ou…